<a href="https://colab.research.google.com/github/shannonfernandes25/new_learnings_bioinformatics-/blob/main/Docking_9IM4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install Biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.6 MB/s eta 0:00:00


In [3]:
! pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 45.5 MB/s eta 0:00:00


In [4]:
import os
import numpy as np
import pandas as pd
import requests           # Allows sending HTTP requests (e.g., downloading data from online databases such as PDB, UniProt, or PubChem)
from Bio.PDB import PDBList
from rdkit import Chem                         # Core RDKit module for handling molecules, reading/writing chemical formats
from rdkit.Chem import Descriptors, AllChem    # Descriptors: calculate molecular properties;
                                               # AllChem: advanced chemistry functions (e.g., conformer generation, fingerprints)
import joblib   # Used for saving/loading Python objects (e.g., trained ML models)
                # and parallel processing for speeding up computations

In [5]:
! pip install chembl-webresource-client   # Installs the ChEMBL client to query the ChEMBL bioactivity database programmatically

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 981.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 4.4 MB/s eta 0:00:00


In [21]:
from chembl_webresource_client.new_client import new_client

# Initialize ChEMBL client
target_client = new_client.target
activity_client = new_client.activity

# Step 1: Search for the target by keyword
search_term = "Leukemia drugs"
targets = target_client.search(search_term)

# Display matching targets
print("Available targets for the search term:")
for target in targets:
    print(f"Target: {target['pref_name']}, ChEMBL ID: {target['target_chembl_id']}")

# Step 2: Choose a target (e.g., the first match)
target_id = targets[0]['target_chembl_id']  # Automatically use the first result
print(f"Selected Target: {targets[0]['pref_name']} ({target_id})")

# Step 3: Fetch inhibitors for the selected target
activities = activity_client.filter(target_chembl_id=target_id, standard_type="IC50").only(
    ["molecule_chembl_id", "canonical_smiles", "standard_value"]
)

# Convert to a Pandas DataFrame
data = []
for activity in activities:
    smiles = activity.get("canonical_smiles")
    ic50 = activity.get("standard_value")
    molecule_id = activity.get("molecule_chembl_id")
    if smiles and ic50:            #Skips entries that don’t have SMILES or IC50 values.
        data.append({"Molecule ChEMBL ID": molecule_id, "SMILES": smiles, "IC50 (nM)": ic50})

df = pd.DataFrame(data)

# Step 4: Save the data to a CSV file
output_file = "Leukemia_compounds.csv"
df.to_csv(output_file, index=False)
print(f"Inhibitors saved to {output_file}")

# Display the first few inhibitors
print(df.head())

Available targets for the search term:
Target: Pleiotropic ABC efflux transporter of multiple drugs, ChEMBL ID: CHEMBL1697658
Target: Myeloid leukemia factor 2, ChEMBL ID: CHEMBL4295830
Target: Leukemia cell, ChEMBL ID: CHEMBL614844
Target: Murine leukemia virus, ChEMBL ID: CHEMBL613746
Target: Feline leukemia virus, ChEMBL ID: CHEMBL612745
Target: Sodium-dependent phosphate transporter 1, ChEMBL ID: CHEMBL4295909
Target: Histone acetyltransferase KAT6A, ChEMBL ID: CHEMBL4523382
Target: Friend leukemia integration 1 transcription factor, ChEMBL ID: CHEMBL5465299
Target: Thrombopoietin receptor, ChEMBL ID: CHEMBL1864
Target: Cytochrome P450 2D6, ChEMBL ID: CHEMBL289
Target: Moloney murine leukemia virus, ChEMBL ID: CHEMBL613500
Target: Rauscher murine leukemia virus, ChEMBL ID: CHEMBL613748
Target: Friend leukemia cell line, ChEMBL ID: CHEMBL614499
Target: Leukemia 60 cell line, ChEMBL ID: CHEMBL614014
Target: Induced myeloid leukemia cell differentiation protein Mcl-1 homolog, ChEMBL I